# Ray Pi Estimation - Assignment 3

This notebook demonstrates distributed Pi estimation using Ray framework.

## Setup:
- **Local Cluster:** 1 master + 1 worker (2 cores, 4GB RAM each)
- **Total Resources:** 4 cores available for computation

In [ ]:
import ray
import time

## Constants and Configuration

- `FIXED_STEPS`: Number of steps for core scaling experiments (1,000,000)
- `STEP_COUNTS`: Different step counts for step scaling experiments

In [ ]:
FIXED_STEPS = 1000000
STEP_COUNTS = [1000, 10000, 100000, 1000000]

## Ray Remote Function

The `@ray.remote` decorator makes this function executable on Ray workers.

**Riemann Sum Method:**
- Approximates π using the integral: ∫₀¹ 4/(1+x²) dx = π
- Each worker calculates a chunk of rectangles under the curve

In [ ]:
@ray.remote
def calculate_pi_chunk(start_index, chunk_size, total_steps_n):
    """
    Calculate a chunk of the Pi estimation using Riemann sums.
    
    Args:
        start_index: Starting index for this chunk
        chunk_size: Number of steps to calculate
        total_steps_n: Total number of steps (for delta_x calculation)
    
    Returns:
        Partial sum for this chunk
    """
    total_area = 0.0
    delta_x = 1.0 / total_steps_n  # Width of each rectangle
    
    for index in range(start_index, start_index + chunk_size):
        x = delta_x * (index - 0.5)  # Midpoint of rectangle
        area = 4.0 / (1.0 + x * x) * delta_x  # Height * width
        total_area += area
        
    return total_area

## Experiment Runner

Distributes work across Ray workers and collects results.

In [ ]:
def run_ray_experiment(total_steps, num_cores):
    """
    Run Pi estimation experiment using Ray.
    
    Args:
        total_steps: Number of Riemann sum steps
        num_cores: Number of parallel workers to use
    
    Returns:
        runtime: Execution time in seconds
        estimated_pi: Calculated value of π
    """
    start_time = time.time()
    
    # Divide work evenly among workers
    num_tasks = num_cores
    chunk_size = total_steps // num_tasks
    remainder = total_steps % num_tasks
    
    # Submit tasks to Ray workers
    futures = []
    current_index = 1
    for i in range(num_tasks):
        current_chunk_size = chunk_size + (1 if i < remainder else 0)
        
        # .remote() submits task asynchronously
        future = calculate_pi_chunk.remote(current_index, current_chunk_size, total_steps)
        futures.append(future)
        
        current_index += current_chunk_size
    
    # ray.get() waits for all tasks to complete and retrieves results
    results = ray.get(futures)
    estimated_pi = sum(results)
    
    runtime = time.time() - start_time
    
    return runtime, estimated_pi

## Connect to Ray Cluster

Connect to the existing Ray cluster (1 master + 1 worker = 4 cores total).

In [ ]:
# Connect to Ray cluster
ray.init(address="auto")

available_cores = int(ray.available_resources().get("CPU", 0))
print(f"Connected to Ray Cluster")
print(f"Available CPU cores: {available_cores}")

## Set Total Cores for Experiments

Determine how many cores to use for the experiments.

In [ ]:
total_cores = available_cores

if total_cores == 0:
    print("Error: No CPU resources detected by Ray.")
else:
    print(f"Running experiments with {total_cores} cores")

## Experiment 1: Core Scaling

Test performance with the available cores (fixed steps = 1,000,000).

This experiment shows how well the computation scales with the number of cores.

In [ ]:
print("=" * 80)
print("Core Scaling Experiment")
print("=" * 80)

core_results = []

runtime, pi = run_ray_experiment(FIXED_STEPS, total_cores)
result_key = f"{total_cores}_cores"

print(f"\nKey: {result_key}")
print(f"Steps: {FIXED_STEPS:,}")
print(f"Time: {runtime:.4f} seconds")
print(f"Estimated Pi: {pi:.10f}")
print(f"Error from actual Pi: {abs(pi - 3.141592653589793):.10f}")

core_results.append({
    'Key': result_key,
    'Cores': total_cores,
    'Steps': FIXED_STEPS,
    'Time': runtime,
    'Pi': pi
})

## Experiment 2: Step Scaling

Test how execution time changes with different numbers of steps (fixed cores = all available).

This experiment shows how runtime grows with problem size and reveals any overhead for small computations.

In [ ]:
print("\n" + "=" * 80)
print(f"Step Scaling Experiment (Fixed Cores: {total_cores})")
print("=" * 80)

step_results = []

for steps in STEP_COUNTS:
    runtime, pi = run_ray_experiment(steps, total_cores)
    
    print(f"\nSteps: {steps:>7,}")
    print(f"Time: {runtime:.4f} seconds")
    print(f"Estimated Pi: {pi:.10f}")
    
    step_results.append({
        'Key': str(steps),
        'Steps': steps,
        'Time': runtime,
        'Pi': pi
    })

## Final Results Summary

In [ ]:
print("\n" + "=" * 80)
print("FINAL RESULTS SUMMARY")
print("=" * 80)

print("\n--- Core Scaling Results ---")
for res in core_results:
    print(f"Cores: {res['Cores']}, Steps: {res['Steps']:,}, Time: {res['Time']:.4f}s, Pi: {res['Pi']:.10f}")

print("\n--- Step Scaling Results ---")
for res in step_results:
    print(f"Steps: {res['Steps']:>7,}, Time: {res['Time']:.4f}s, Pi: {res['Pi']:.10f}")

## Cleanup

Disconnect from the Ray cluster.

In [ ]:
ray.shutdown()
print("Disconnected from Ray cluster.")